<a href="https://colab.research.google.com/github/pgurazada/mlops-workshops/blob/main/advanced-python-course/python-for-machine-learning/machine_failure_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective

To illustrate a supervised learning workflow for classification tasks.


# Setup

In [1]:
import sklearn
import joblib

from sklearn.datasets import fetch_openml

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split, RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [2]:
sklearn.set_config(display='diagram')

# Business Context

For this session consider the case of predicting machinery failure based on the quality of the machinery and its wear and tear. Such predictions often require an expert intervention on the shopfloor and result in high dependencies on the expert. Predicting machine failure in advance based on the current state of the machinery can avoid large costs inncurred due to unplanned maintenance breakdowns.

The dataset used in this session is reflective of actual production data and is hosted on Open ML.


# Data

In [3]:
dataset = fetch_openml(data_id=42890, as_frame=True, parser="auto")

In [4]:
data_df = dataset.data

In [5]:
target = 'Machine failure'
numeric_features = [
    'Air temperature [K]',
    'Process temperature [K]',
    'Rotational speed [rpm]',
    'Torque [Nm]',
    'Tool wear [min]'
]
categorical_features = ['Type']

## Attribute Information:

The dataset consists of 10 000 data points stored as rows with 14 features in columns

- UID: unique identifier ranging from 1 to 10000
- product ID: consisting of a letter L, M, or H for low (50% of all products), medium (30%) and high (20%) as product quality variants and a variant-specific serial number
- air temperature [K]: generated using a random walk process later normalized to a standard deviation of 2 K around 300 K
- process temperature [K]: generated using a random walk process normalized to a standard deviation of 1 K, added to the air temperature plus 10 K.
- rotational speed [rpm]: calculated from a power of 2860 W, overlaid with a normally distributed noise
- torque [Nm]: torque values are normally distributed around 40 Nm with a f = 10 Nm and no negative values.
- tool wear [min]: The quality variants H/M/L add 5/3/2 minutes of tool wear to the used tool in the process.
- 'machine failure' label that indicates, whether the machine has failed in this particular datapoint for any of the following failure modes are true.

The machine failure consists of five independent failure modes
tool wear failure (TWF): the tool will be replaced of fail at a randomly selected tool wear time between 200-240 mins (120 times in our dataset). At this point in time, the tool is replaced 69 times, and fails 51 times (randomly assigned).

- heat dissipation failure (HDF): heat dissipation causes a process failure, if the difference between air- and process temperature is below 8.6 K and the tools rotational speed is below 1380 rpm. This is the case for 115 data points.
- power failure (PWF): the product of torque and rotational speed (in rad/s) equals the power required for the process. If this power is below 3500 W or above 9000 W, the process fails, which is the case 95 times in our dataset.
- overstrain failure (OSF): if the product of tool wear and torque exceeds 11,000 minNm for the L product variant (12,000 M, 13,000 H), the process fails due to overstrain. This is true for 98 datapoints.
- random failures (RNF): each process has a chance of 0,1 % to fail regardless of its process parameters. This is the case for only 5 datapoints, less than could be expected for 10,000 datapoints in our dataset.

If at least one of the above failure modes is true, the process fails and the 'machine failure' label is set to 1.

# EDA

In [6]:
data_df[numeric_features].describe()

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min]
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,300.004930,310.005560,1538.776100,39.986910,107.951000
std,2.000259,1.483734,179.284096,9.968934,63.654147
min,295.300000,305.700000,1168.000000,3.800000,0.000000
25%,298.300000,308.800000,1423.000000,33.200000,53.000000
50%,300.100000,310.100000,1503.000000,40.100000,108.000000
75%,301.500000,311.100000,1612.000000,46.800000,162.000000
max,304.500000,313.800000,2886.000000,76.600000,253.000000


In [7]:
data_df[categorical_features].describe()

,Type
count,10000
unique,3
top,L
freq,6000


In [8]:
data_df[target].value_counts()

Machine failure
0    9661
1     339
Name: count, dtype: int64

# Model Estimation

In [9]:
X = data_df[numeric_features + categorical_features]
y = data_df[target]

In [10]:
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

In [11]:
preprocessor = make_column_transformer(
    (StandardScaler(), numeric_features),
    (OneHotEncoder(handle_unknown='ignore'), categorical_features)
)

In [12]:
model_logistic_regression = LogisticRegression(n_jobs=-1)

In [13]:
model_pipeline = make_pipeline(
    preprocessor,
    model_logistic_regression
)

In [14]:
model_pipeline.fit(Xtrain, ytrain)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['Air temperature [K]',
                                                   'Process temperature [K]',
                                                   'Rotational speed [rpm]',
                                                   'Torque [Nm]',
                                                   'Tool wear [min]']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Type'])])),
                ('logisticregression', LogisticRegression(n_jobs=-1))])

# Model Evaluation

In [15]:
model_pipeline.predict(Xtest)

array([0, 0, 0, ..., 0, 0, 0])

In [16]:
accuracy_score(ytest, model_pipeline.predict(Xtest))

0.9735

In [17]:
print(classification_report(ytest, model_pipeline.predict(Xtest)))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1939
           1       0.67      0.26      0.38        61

    accuracy                           0.97      2000
   macro avg       0.82      0.63      0.68      2000
weighted avg       0.97      0.97      0.97      2000



# Hyperparameter Tuning

In [18]:
preprocessor = make_column_transformer(
    (StandardScaler(), numeric_features),
    (OneHotEncoder(handle_unknown='ignore'), categorical_features)
)

In [19]:
model_logistic_regression = LogisticRegression(n_jobs=-1)

In [20]:
model_pipeline = make_pipeline(
    preprocessor,
    model_logistic_regression
)

In [21]:
model_pipeline.named_steps

{'columntransformer': ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                  ['Air temperature [K]',
                                   'Process temperature [K]',
                                   'Rotational speed [rpm]', 'Torque [Nm]',
                                   'Tool wear [min]']),
                                 ('onehotencoder',
                                  OneHotEncoder(handle_unknown='ignore'),
                                  ['Type'])]),
 'logisticregression': LogisticRegression(n_jobs=-1)}

In [22]:
param_distribution = {
    "logisticregression__C": [0.001, 0.01, 0.1, 0.5, 1, 5, 10]
}

In [23]:
rand_search_cv = RandomizedSearchCV(
    model_pipeline,
    param_distribution,
    n_iter=3,
    cv=3,
    random_state=42
)

In [24]:
rand_search_cv.fit(Xtrain, ytrain)

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('standardscaler',
                                                                               StandardScaler(),
                                                                               ['Air '
                                                                                'temperature '
                                                                                '[K]',
                                                                                'Process '
                                                                                'temperature '
                                                                                '[K]',
                                                                                'Rotational '
                                                                                'speed '
                                                                                '[rpm]',
                                                                                'Torque '
                                                                                '[Nm]',
                                                                                'Tool '
                                                                                'wear '
                                                                                '[min]']),
                                                                              ('onehotencoder',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               ['Type'])])),
                                             ('logisticregression',
                                              LogisticRegression(n_jobs=-1))]),
                   n_iter=3,
                   param_distributions={'logisticregression__C': [0.001, 0.01,
                                                                  0.1, 0.5, 1,
                                                                  5, 10]},
                   random_state=42)

In [25]:
rand_search_cv.best_estimator_

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['Air temperature [K]',
                                                   'Process temperature [K]',
                                                   'Rotational speed [rpm]',
                                                   'Torque [Nm]',
                                                   'Tool wear [min]']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Type'])])),
                ('logisticregression', LogisticRegression(C=5, n_jobs=-1))])

In [26]:
rand_search_cv.best_score_

0.9691250146619894

In [27]:
print(classification_report(ytest, rand_search_cv.best_estimator_.predict(Xtest)))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      1939
           1       0.63      0.28      0.39        61

    accuracy                           0.97      2000
   macro avg       0.80      0.64      0.69      2000
weighted avg       0.97      0.97      0.97      2000



# Serialization

In [28]:
!pip show scikit-learn

Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: imbalanced-learn


In [29]:
%%writefile requirements.txt
scikit-learn==1.2.2

Writing requirements.txt


In [30]:
%%writefile train.py

import joblib

from sklearn.datasets import fetch_openml

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split, RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

dataset = fetch_openml(data_id=42890, as_frame=True, parser="auto")

data_df = dataset.data

target = 'Machine failure'
numeric_features = [
    'Air temperature [K]',
    'Process temperature [K]',
    'Rotational speed [rpm]',
    'Torque [Nm]',
    'Tool wear [min]'
]
categorical_features = ['Type']

print("Creating data subsets")

X = data_df[numeric_features + categorical_features]
y = data_df[target]

Xtrain, Xtest, ytrain, ytest = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

preprocessor = make_column_transformer(
    (StandardScaler(), numeric_features),
    (OneHotEncoder(handle_unknown='ignore'), categorical_features)
)

model_logistic_regression = LogisticRegression(n_jobs=-1)

print("Estimating Best Model Pipeline")

model_pipeline = make_pipeline(
    preprocessor,
    model_logistic_regression
)

param_distribution = {
    "logisticregression__C": [0.001, 0.01, 0.1, 0.5, 1, 5, 10]
}

rand_search_cv = RandomizedSearchCV(
    model_pipeline,
    param_distribution,
    n_iter=3,
    cv=3,
    random_state=42
)

rand_search_cv.fit(Xtrain, ytrain)

print("Logging Metrics")
print(f"Accuracy: {rand_search_cv.best_score_}")

print("Serializing Model")

saved_model_path = "model.joblib"

joblib.dump(rand_search_cv.best_estimator_, saved_model_path)

Writing train.py


In [31]:
!python train.py

Creating data subsets
Estimating Best Model Pipeline
Logging Metrics
Accuracy: 0.9691250146619894
Serializing Model


# Test Predictions

In [32]:
saved_model = joblib.load("model.joblib")

In [33]:
saved_model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['Air temperature [K]',
                                                   'Process temperature [K]',
                                                   'Rotational speed [rpm]',
                                                   'Torque [Nm]',
                                                   'Tool wear [min]']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Type'])])),
                ('logisticregression', LogisticRegression(C=5, n_jobs=-1))])

In [34]:
saved_model.predict(Xtest)

array([0, 0, 0, ..., 0, 0, 0])